<a href="https://colab.research.google.com/github/YeChen/RevealAPITest/blob/main/reveal_api_test_17_wordlists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import os
from google.colab import userdata
from IPython.display import display, HTML

# Step 1: Retrieve USER, PASSWORD, and case_id secrets from Colab
username = userdata.get("DEMO_USER")
password = userdata.get("DEMO_PASSWORD")
case_id = userdata.get("API_STANDARD_PROJECTID")  # Renaming API_STANDARD_PROJECTID to case_id

if not username or not password or not case_id:
    raise ValueError("Secrets USER, PASSWORD, and case_id must be set in Colab.")

def print_color(message, color):
    display(HTML(f'<p style="color: {color};">{message}</p>'))

# Step 2: Define the authentication function
def authenticate(username, password):
    """
    Authenticates with the Reveal API using username and password.
    Returns loginSessionId and userId if authentication is successful.
    """
    login_url = "https://consulting.us-east-1.reveal11.cloud/rest/api/v2/login"
    login_data = {
        "username": username,
        "password": password
    }

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Reveal-API-Tester/1.0"
    }

    response = requests.post(login_url, json=login_data, headers=headers)
    response.raise_for_status()  # Raise an error if the request fails

    login_response = response.json()
    login_session_id = login_response.get("loginSessionId")
    user_id = login_response.get("userId")

    if not login_session_id or not user_id:
        raise ValueError("Failed to retrieve loginSessionId and userId from login response.")

    print("Authentication successful!")
    return login_session_id, user_id

# Step 3: Define the function to get hit reports
def get_hit_reports(login_session_id, case_id):
    """
    Retrieves a list of hit reports for the specified caseId, including terms under each name.
    """
    hit_report_url = f"https://consulting.us-east-1.reveal11.cloud/rest/api/v2/{case_id}/jobs/hitreport"

    headers = {
        "Authorization": f"Bearer {login_session_id}",
        "incontrolauthtoken": login_session_id,
        "User-Agent": "Reveal-API-Tester/1.0",
        "Accept": "application/json"
    }

    response = requests.get(hit_report_url, headers=headers)
    response.raise_for_status()

    hit_reports_response = response.json()

    # Check if the API response contains results
    hit_reports = hit_reports_response.get("results", hit_reports_response)

    if isinstance(hit_reports, list):
        print("Hit Reports:")
        for report in hit_reports:
            report_name = report.get("name", "Unnamed Report")
            print(f"Report Name: {report_name}")

            # Extract terms if they exist
            terms = report.get("terms", [])
            if terms:
                print("  Terms:")
                for term in terms:
                    print(f"    - {term}")
            else:
                print("  No terms found for this report.")
    else:
        print("Unexpected data structure for hit reports:", hit_reports)

# Main Execution
try:
    # Step 1: Authenticate to get session ID and user ID
    session_id, user_id = authenticate(username, password)

    # Step 2: Retrieve and print hit reports and terms for the specified case_id
    get_hit_reports(session_id, case_id)

except requests.exceptions.RequestException as e:
    print_color(f"API request failed: {e}", "red")
except ValueError as e:
    print_color(f"Error: {e}", "red")


Authentication successful!
Hit Reports:
Report Name: API_TEST_FROMSWAGGER
  Terms:
    - {'termId': '4', 'term': 'SWAGGER'}
Report Name: API_TEST_MONEY
  Terms:
    - {'termId': '2', 'term': 'currency'}
    - {'termId': '3', 'term': 'dolladollabillyall'}
    - {'termId': '1', 'term': 'money'}
